<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">Colab KSO Tutorials #2: Upload new footage</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Aug 19th, 2022</h5>

# Set up and requirements

## Install kso_data_management and its requirements

In [ ]:
try:
  import google.colab
  import os
  from IPython.display import clear_output
  IN_COLAB = True
  print("Running in Colab...")

  # Clone koster_data_management repo
  !git clone --quiet --recurse-submodules -b main https://github.com/ocean-data-factory-sweden/koster_data_management.git
  !pip install -q --upgrade pip
  !pip install -q -r koster_data_management/requirements.txt
  
  # Fix libmagic issue
  !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

  # Enable external widgets
  from google.colab import output
  output.enable_custom_widget_manager()

  os.chdir("koster_data_management/tutorials")
  try:
    clear_output()
    print("All packages are installed and ready to go!")
  except:
    clear_output()
    print("There have been some issues installing the packages!")
except:
  IN_COLAB = False
  import sys
  if "ipysheet" not in sys.modules:
      !pip install -q --no-warn-script-location --upgrade pip
      !pip install -q ipysheet==0.4.4
      
  print("Running locally... you're good to go!")

### Import Python packages

In [ ]:
# Set the directory of the libraries
import sys, os
from pathlib import Path

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Specify the path of the tutorials
os.chdir("koster_data_management/tutorials")
sys.path.append('..')


# Enable third party widgets
from google.colab import output
output.enable_custom_widget_manager()

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t2_utils as t2
import kso_utils.project_utils as p_utils
#import utils.server_utils as serv_utils

print("Packages loaded successfully")

## Choose your project

In [ ]:
project_name = t_utils.choose_project()

## Initiate sql and get server or local storage details

In [ ]:
# Save project name
project = p_utils.find_project(project_name=project_name.value)
# Initiate db
db_info_dict = t_utils.initiate_db(project)

# Select the survey linked to the videos

In [ ]:
survey_i = t2.select_survey(db_info_dict)

In [ ]:
survey_name = t2.confirm_survey(survey_i, 
                                db_info_dict)

## Select new deployments

To save time you can select multiple deployments **recorded on the same day**

In [ ]:
deployment_selected, survey_row, survey_server_name = t2.select_deployment(
    project = project,
    db_info_dict = db_info_dict,
    survey_i = survey_i
)

## Specify the date of the deployments

In [ ]:
deployment_date = t2.select_eventdate(survey_row)

## Check the database to avoid deployment duplicates

In [ ]:
deployment_names = t2.check_deployment(
    deployment_selected, 
    deployment_date, 
    survey_server_name, 
    db_info_dict, 
    survey_i
)

## Update new deployment files

In [ ]:
movie_files_server = t2.update_new_deployments(
    deployment_filenames = deployment_names, 
    db_info_dict = db_info_dict, 
    survey_server_name = survey_server_name,
    deployment_date = deployment_date
)

## Specify deployment details

In [ ]:
deployment_info = t2.record_deployment_info(
    deployment_names, 
    db_info_dict
)

## Review deployment  details

In [ ]:
new_deployment_row = t2.confirm_deployment_details(
    deployment_names, 
    survey_server_name, 
    db_info_dict, 
    survey_i, 
    deployment_info,
    movie_files_server,
    deployment_date
)

# !!!Only pass this point if deployment details are correct!!!

### Update movies csv and upload video to s3

In [ ]:
t2.upload_concat_movie(
    db_info_dict,
    new_deployment_row
)

In [ ]:
#END